# Goodhart's Law Environment Test

This notebook verifies that the environment is correctly set up.

In [ ]:
import torch
import sys
print(f"Python version: {sys.version}")
print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

In [ ]:
# Test goodharts package imports
from goodharts.utils.device import get_device
from goodharts.config import get_config

device = get_device()
config = get_config()

print(f"Device: {device}")
print(f"Config loaded: world_size={config.get('WORLD_SIZE')}, view_size={config.get('VIEW_SIZE')}")
print("Package imports successful.")

In [ ]:
# Quick CLI sanity check
!python -m goodharts.training.train_ppo --help